# 경사 하강법을 이용한 얕은 신경망 학습  
데이터 셋을 만들고, trainable parameter도 텐서플러우를 이용해 학습하는 과정을 진행해 보겠습니다.    
텐서플러우 이용해서 얕은 신경망을 구현하고 경사하강법을 이용해 학습해 보겠습니다.

In [53]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정  
1000 epochs 까지 학습을 해보겠습니다.

In [54]:
EPOCHS = 1000

## 네트워크 구조 정의  
이번엔 numpy가 아닌 tensorflow를 이용할 것입니다.
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)  
-> 총 10개의 class 중에 하나로 들어갈 것입니다.  
* 텐서플러우 2.0의 네트워크 구조하기 위해서는 케라스의 모델을 상속해서 클래스를 구현합니다.  
* 상속을 했을 경우에는 상속을 한 initialize를 잊지 말고 해야합니다.  
* 클래스의 모든 메서드들은 처음 파라메터를 self 로 받아주어야 합니다.

In [55]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1=tf.keras.layers.Dense(128,input_dim=2,activation='sigmoid')
        self.d2=tf.keras.layers.Dense(10,activation='softmax')
        
    def call(self,x,training=None, mask=None):
        x=self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [70]:
@tf.function
def train_step(model,inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        prediction= model(inputs)
        loss=loss_object(labels, predictions)
    gradients=tape.gradient(loss,model.trainable_variables) #df(x)/dx
    
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    train_loss(loss)
    train_metric(labels,predictions)
    

## 데이터셋 생성, 전처리  
랜덤시드를 고정해주면 여러번 시도해도 랜덤값은 항상 동일하게 된다.  
센터포인트들이 중간에 둥글게 퍼지게 될겁니다.

In [71]:
np.random.seed(0)

pts=list() #길이가 2
labels=list() #10개의 class 중에 하나 0~9
center_pts=np.random.uniform(-8.0,8.0,(10,2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)
pts=np.stack(pts,axis=0).astype(np.float32)
labels=np.stack(labels, axis=0) 
train_ds=tf.data.Dataset.from_tensor_slices((pts,labels)).shuffle(1000).batch(32)

## 모델 생성

In [72]:
model=MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [73]:
loss_ojbect=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [74]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프  
32개의 입력과 32개의 정답이 계속해서 나오게 됩니다.  
매 에포크마다 데이터셋에서 입력이랑 정답받아서 트레인셋 돌려준다. 그리고 학습하는 과정을 출력해 준다. 그래서 Epoch, loss, Accuracy를 출력한다.

In [76]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model,x,label,loss_object,optimizer,train_loss,train_accuracy)
        
    template= 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100))

NameError: name 'loss_object' is not defined

## 데이터셋 및 학습 파라미터 저장

In [77]:
np.savez_compressed('ch2_dataset.npz',inputs=pts,labels=labels)

W_h,b_h=model.d1.get_weights()
W_o,b_o=model.d2.get_weights()
W_h =np.transpose(W_h)
W_o=np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                   W_h=W_h,
                   b_h=b_h,
                   W_o=W_o,
                   b_o=b_o)